<a href="https://colab.research.google.com/github/MichaelHMorei/arquivos_de_apoio/blob/main/Igtipy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://apache.osuosl.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz
!tar xf spark-3.2.1-bin-hadoop2.7.tgz

In [2]:
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop2.7"

In [3]:
os.environ["SPARK_HOME"]

'/content/spark-3.2.1-bin-hadoop2.7'

In [4]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [5]:
pip install spark-nlp==2.4.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 108 kB 5.0 MB/s 


In [6]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  2.4.2
Apache Spark version:  3.2.1


In [7]:
import pandas as pd
import numpy as np
import os
import zipfile
import requests
from io import BytesIO
#Importando bibliotecas
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pyspark
from pyspark.sql.types import *
from datetime import *
from pyspark import SparkConf, SparkContext

In [8]:
#Site contendo os dados
url = "https://download.inep.gov.br/microdados/microdados_enem_2020.zip"

In [9]:
#Criando pasta para armazenar os dados
os.makedirs('./enadade2020',exist_ok = True)

In [10]:
#Fazendo o download do conteudo
file =  BytesIO(
     requests.get(url).content)
 
# Extraindo o conteudo do arquivo zip na pasta criada.
zip = zipfile.ZipFile(file)
zip.extractall("./enadade2020")

In [12]:
df_M = spark.read.option("encoding", "cp1252").csv("./enadade2020/DADOS/MICRODADOS_ENEM_2020.csv",inferSchema = 'True', sep =";", header ='True') 

In [13]:
df_M.show(10)

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+-------------------+---------+---------+----------------------+------------------+---------------+------------------+-------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|T

In [14]:
df_M.createOrReplaceTempView('df_enada')

In [53]:
retur = spark.sql("""
  select df_enada.SG_UF_PROVA, df_enada.NU_NOTA_MT , df_enada.NU_NOTA_LC , df_enada.NU_NOTA_CH , df_enada.TP_SEXO , df_enada.NO_MUNICIPIO_ESC , df_enada.Q016 , df_enada.Q002, df_enada.Q005 , df_enada.Q001, df_enada.Q006, df_enada.TP_LINGUA , df_enada.TP_FAIXA_ETARIA , df_enada.TP_ESCOLA from df_enada
    where df_enada.SG_UF_PROVA = 'MG'
    AND df_enada.SG_UF_ESC = 'MG'
""")


In [ ]:
retur.select('SG_UF_PROVA','NU_NOTA_MT').filter(col('NU_NOTA_MT') > 0).groupBy("SG_UF_PROVA").agg(mean("NU_NOTA_MT")).show()

+-----------+-----------------+
|SG_UF_PROVA|  avg(NU_NOTA_MT)|
+-----------+-----------------+
|         MG|579.9392153496156|
+-----------+-----------------+



In [ ]:
retur.select('SG_UF_PROVA','df_enada.NU_NOTA_CH').filter(col('NU_NOTA_LC') > 0).groupBy("NU_NOTA_CH").agg(mean("NU_NOTA_LC")).show()

+-----------+-----------------+
|SG_UF_PROVA|  avg(NU_NOTA_LC)|
+-----------+-----------------+
|         MG|553.4278863933847|
+-----------+-----------------+



In [ ]:
retur.select('SG_UF_PROVA','NU_NOTA_CH','TP_SEXO').filter(col('NU_NOTA_CH') > 0).filter(col('TP_SEXO') == 'F').groupBy("SG_UF_PROVA").agg(mean("NU_NOTA_CH")).show()

+-----------+-----------------+
|SG_UF_PROVA|  avg(NU_NOTA_CH)|
+-----------+-----------------+
|         MG|541.9433991568674|
+-----------+-----------------+



In [ ]:
retur.select('SG_UF_PROVA','NU_NOTA_CH','TP_SEXO').filter(col('NU_NOTA_CH') > 0).filter(col('TP_SEXO') == 'M').groupBy("SG_UF_PROVA").agg(mean("NU_NOTA_CH")).show()

+-----------+-----------------+
|SG_UF_PROVA|  avg(NU_NOTA_CH)|
+-----------+-----------------+
|         MG|560.3305497531334|
+-----------+-----------------+



In [ ]:
retur.select('SG_UF_PROVA','NU_NOTA_MT','TP_SEXO', 'NO_MUNICIPIO_ESC').filter(col('NU_NOTA_MT') > 0).filter(col('TP_SEXO') == 'F').filter(col('NO_MUNICIPIO_ESC') == 'Montes Claros').groupBy("SG_UF_PROVA").agg(mean("NU_NOTA_MT")).show()

+-----------+-----------------+
|SG_UF_PROVA|  avg(NU_NOTA_MT)|
+-----------+-----------------+
|         MG|563.3175837320574|
+-----------+-----------------+



In [ ]:
retur.select('SG_UF_PROVA','NU_NOTA_MT','Q021', 'NO_MUNICIPIO_ESC').filter(col('NU_NOTA_MT') > 0).filter(col('Q021') == 'B').filter(col('NO_MUNICIPIO_ESC') == 'Sabará').groupBy("SG_UF_PROVA").agg(mean("NU_NOTA_MT")).show()

+-----------+-----------------+
|SG_UF_PROVA|  avg(NU_NOTA_MT)|
+-----------+-----------------+
|         MG|580.1029850746269|
+-----------+-----------------+



In [18]:
retur.select('SG_UF_PROVA','NU_NOTA_CH','Q016').filter(col('NU_NOTA_CH') > 0).filter(col('Q016') == 'C').groupBy("SG_UF_PROVA").agg(mean("NU_NOTA_CH")).show()

+-----------+-----------------+
|SG_UF_PROVA|  avg(NU_NOTA_CH)|
+-----------+-----------------+
|         MG|601.0198979591837|
+-----------+-----------------+



In [20]:
retur.select('SG_UF_PROVA','NU_NOTA_MT','Q002').filter(col('NU_NOTA_MT') > 0).filter(col('Q002') == 'G').groupBy("SG_UF_PROVA").agg(mean("NU_NOTA_MT")).show()

+-----------+-----------------+
|SG_UF_PROVA|  avg(NU_NOTA_MT)|
+-----------+-----------------+
|         MG|642.4147184604419|
+-----------+-----------------+



In [25]:
retur.select('SG_UF_PROVA','NU_NOTA_MT', 'NO_MUNICIPIO_ESC').filter(col('NU_NOTA_MT') > 0).filter((col('NO_MUNICIPIO_ESC') == 'Belo Horizonte') | (col('NO_MUNICIPIO_ESC') == 'Conselheiro Lafaiete')).groupBy("SG_UF_PROVA").agg(mean("NU_NOTA_MT")).show()

+-----------+-----------------+
|SG_UF_PROVA|  avg(NU_NOTA_MT)|
+-----------+-----------------+
|         MG|621.0512265016049|
+-----------+-----------------+



In [30]:
retur.select('SG_UF_PROVA','NU_NOTA_CH','Q005').filter(col('NU_NOTA_CH') > 0).filter(col('Q005') == 1).groupBy("SG_UF_PROVA").agg(mean("NU_NOTA_CH")).show()

+-----------+-----------------+
|SG_UF_PROVA|  avg(NU_NOTA_CH)|
+-----------+-----------------+
|         MG|538.9960784313726|
+-----------+-----------------+



In [36]:
retur.select('SG_UF_PROVA','NU_NOTA_CH','Q001', 'Q006').filter(col('NU_NOTA_CH') > 0).filter((col('Q001') == 'G') & (col('Q006') == 'L')).groupBy("SG_UF_PROVA").agg(mean("NU_NOTA_CH")).show()

+-----------+-----------------+
|SG_UF_PROVA|  avg(NU_NOTA_CH)|
+-----------+-----------------+
|         MG|627.1642857142857|
+-----------+-----------------+



In [47]:
retur.select('SG_UF_PROVA','NU_NOTA_MT','TP_SEXO','TP_LINGUA').filter(col('NU_NOTA_MT') > 0).filter((col('TP_SEXO') == 'F') & (col('TP_LINGUA') == 1) & (col('NO_MUNICIPIO_ESC') == 'Lavras')).groupBy("SG_UF_PROVA").agg(mean("NU_NOTA_MT")).show()

+-----------+-----------------+
|SG_UF_PROVA|  avg(NU_NOTA_MT)|
+-----------+-----------------+
|         MG|513.3833333333334|
+-----------+-----------------+



In [49]:
retur.select('SG_UF_PROVA','NU_NOTA_MT','TP_SEXO','NO_MUNICIPIO_ESC').filter(col('NU_NOTA_MT') > 0).filter((col('TP_SEXO') == 'M') & (col('NO_MUNICIPIO_ESC') == 'Ouro Preto')).groupBy("SG_UF_PROVA").agg(mean("NU_NOTA_MT")).show()

+-----------+-----------------+
|SG_UF_PROVA|  avg(NU_NOTA_MT)|
+-----------+-----------------+
|         MG|615.5823529411764|
+-----------+-----------------+



In [52]:
retur.select('SG_UF_PROVA','NU_NOTA_CH','TP_FAIXA_ETARIA').filter(col('NU_NOTA_CH') > 0).filter(col('TP_FAIXA_ETARIA') == 11).groupBy("SG_UF_PROVA").agg(mean("NU_NOTA_CH")).show()

+-----------+----------------+
|SG_UF_PROVA| avg(NU_NOTA_CH)|
+-----------+----------------+
|         MG|474.428947368421|
+-----------+----------------+



In [56]:
retur.select('SG_UF_PROVA','NU_NOTA_MT','TP_SEXO','TP_ESCOLA').filter(col('NU_NOTA_MT') > 0).filter((col('TP_SEXO') == 'F') & (col('TP_ESCOLA') == 3) & (col('NO_MUNICIPIO_ESC') == 'Nova Lima') | (col('NO_MUNICIPIO_ESC') == 'Belo Horizonte') | (col('NO_MUNICIPIO_ESC') == 'Sabará') | (col('NO_MUNICIPIO_ESC') == 'Betim')).groupBy("SG_UF_PROVA").agg(mean("NU_NOTA_MT")).show()

+-----------+-----------------+
|SG_UF_PROVA|  avg(NU_NOTA_MT)|
+-----------+-----------------+
|         MG|619.1268339562199|
+-----------+-----------------+



In [ ]:
TP_ESCOLA

In [29]:
retur.filter(col('Q005') ==1).show()

+-----------+----------+----------+----------+-------+----------------+----+----+----+
|SG_UF_PROVA|NU_NOTA_MT|NU_NOTA_LC|NU_NOTA_CH|TP_SEXO|NO_MUNICIPIO_ESC|Q016|Q002|Q005|
+-----------+----------+----------+----------+-------+----------------+----+----+----+
|         MG|     365.6|     502.6|     410.6|      F|         Nanuque|   A|   H|   1|
|         MG|     689.1|     583.8|     604.2|      F|  Belo Horizonte|   B|   F|   1|
|         MG|      null|      null|      null|      M|          Muriaé|   B|   B|   1|
|         MG|     477.1|     350.5|     527.6|      F|         Nanuque|   A|   B|   1|
|         MG|     776.8|     657.2|     678.6|      F|  Belo Horizonte|   B|   E|   1|
|         MG|     909.3|     617.6|     755.8|      M|  Belo Horizonte|   B|   E|   1|
|         MG|     333.6|     483.7|     441.9|      F|         Itaobim|   A|   H|   1|
|         MG|      null|      null|      null|      F|      Angelândia|   A|   A|   1|
|         MG|     424.2|     614.6|     655